##⚙️ Models Development Phase

🔶 Install the needed dependencies & Impoert the needed libraries.

In [ ]:
!pip install catboost

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/102.4 MB ? eta -:--:--
   ---------------------------------------- 0.4/102.4 MB 7.6 MB/s eta 0:00:14
   ---------------------------------------- 1.0/102.4 MB 9.2 MB/s eta 0:00:12
    --------------------------------------- 1.7/102.4 MB 10.0 MB/s eta 0:00:11
    --------------------------------------- 2.1/102.4 MB 9.5 MB/s eta 0:00:11
    --------------------------------------- 2.1/102.4 MB 9.5 MB/s eta 0:00:11
   - -------------------------------------- 2.8/102.4 MB 7.8 MB/s eta 0:00:13
   - -------------------------------------- 3.6/102.4 MB 8.5 MB/s eta 0:00:12
   - -------------------------------------- 4.2/102.4 MB 8.9 MB/s eta 0:00:12
   - -------------------------------------- 4.2/102.4 MB 8.9 MB/s eta 0:00:12
   - -------------------------------------- 4.9/102.4 MB 8.1 MB/s eta 0:00:13
   -- ------------------------------------- 5.2/102.4 MB 8.2 MB/s eta 

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold, cross_validate
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, make_scorer, precision_score, recall_score, f1_score
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from catboost import CatBoostClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

🔶 Read & View the cleaned dataset.

- This is the original cleaned dataset

In [ ]:
data1 = pd.read_csv('cleaned_dataset.csv')

In [ ]:
data1.shape

(592766, 19)

In [ ]:
data1.head(5)

,index,course_id,userid_DI,registered,viewed,explored,certified,final_cc_cname_DI,LoE_DI,YoB,gender,grade,start_time_DI,last_event_DI,ndays_act,nforum_posts,nchapters_imputed,nevents_imputed,dropout
0,0,HarvardX/CB22x/2013_Spring,MHxPC130442623,1,0,0,0,United States,Bachelor's,1988.0,m,0,2012-12-19,2013-11-17 00:00:00.000000000,9.0,0,3.0,3.0,1
1,1,HarvardX/CS50x/2012,MHxPC130442623,1,1,0,0,United States,Bachelor's,1988.0,m,0,2012-10-15,2012-12-06 03:56:11.943158893,9.0,0,1.0,1.0,1
2,2,HarvardX/CB22x/2013_Spring,MHxPC130275857,1,0,0,0,United States,Bachelor's,1988.0,m,0,2013-02-08,2013-11-17 00:00:00.000000000,16.0,0,3.0,4.0,1
3,3,HarvardX/CS50x/2012,MHxPC130275857,1,0,0,0,United States,Bachelor's,1988.0,m,0,2012-09-17,2012-11-08 03:56:11.943158893,16.0,0,3.0,3.0,1
4,4,HarvardX/ER22x/2013_Spring,MHxPC130275857,1,0,0,0,United States,Bachelor's,1988.0,m,0,2012-12-19,2013-02-09 03:56:11.943158893,16.0,0,3.0,2.0,1


🔶 Begin Modeling.

- In this stage, we will attempt to create a balanced sample from the cleaned dataset. This will gives us a ***perfect scenario*** (reasonable size for faster execution in Google Colab and balanced dataset) for implementation.
- *This method is not completely suitable for real world scenarios!*

In [ ]:
# Attempt 1
# Set your target column name
target_column = 'dropout'

# Separate the two classes
dropouts = data1[data1[target_column] == 1]
non_dropouts = data1[data1[target_column] == 0]

# Take a random sample from the larger class (dropouts) equal to the number of non-dropouts
dropouts_sampled = dropouts.sample(n=len(non_dropouts), random_state=42)

# Combine both groups
balanced_data = pd.concat([dropouts_sampled, non_dropouts])

# Shuffle the dataset
balanced_data = balanced_data.sample(frac=1, random_state=42).reset_index(drop=True)

# check the balance
print(balanced_data[target_column].value_counts())

dropout
0    64823
1    64823
Name: count, dtype: int64


In [ ]:
# Attempt 2
# Sample smaller balanced data
sample_size = 10000

dropouts_sampled = dropouts.sample(n=sample_size, random_state=42)
non_dropouts_sampled = non_dropouts.sample(n=sample_size, random_state=42)

small_balanced_data = pd.concat([dropouts_sampled, non_dropouts_sampled])
small_balanced_data = small_balanced_data.sample(frac=1, random_state=42).reset_index(drop=True)

# check the balance
print(small_balanced_data[target_column].value_counts())

dropout
0    10000
1    10000
Name: count, dtype: int64


In [ ]:
# Attempt 1
balanced_data.shape

(129646, 19)

In [ ]:
# Attempt 2
small_balanced_data.shape

(20000, 19)

In [ ]:
small_balanced_data.head(10)

,index,course_id,userid_DI,registered,viewed,explored,certified,final_cc_cname_DI,LoE_DI,YoB,gender,grade,start_time_DI,last_event_DI,ndays_act,nforum_posts,nchapters_imputed,nevents_imputed,dropout
0,611647,MITx/8.MReV/2013_Summer,MHxPC130177910,1,1,0,0,India,Less than Secondary,1994.0,m,0.03,2013-07-11,2013-08-01 00:00:00.000000000,4.0,0,8.0,422.0,0
1,240955,HarvardX/ER22x/2013_Spring,MHxPC130247938,1,0,0,0,Brazil,Master's,1988.0,m,0,2013-02-10,2013-04-03 03:56:11.943158893,2.0,0,0.0,2.0,1
2,560082,MITx/6.00x/2013_Spring,MHxPC130460965,1,1,0,0,Poland,Secondary,1993.0,m,0,2013-02-12,2013-02-12 00:00:00.000000000,1.0,0,1.0,5.0,1
3,44300,HarvardX/CS50x/2012,MHxPC130412597,1,0,0,0,Germany,Bachelor's,1978.0,m,0,2012-08-13,2012-10-04 03:56:11.943158893,2.0,0,0.0,2.0,1
4,628720,MITx/8.MReV/2013_Summer,MHxPC130249459,1,1,0,0,India,Master's,1986.0,m,0.04,2013-06-27,2013-07-08 00:00:00.000000000,9.0,0,10.0,1556.0,0
5,486297,MITx/6.00x/2012_Fall,MHxPC130124754,1,1,1,0,United States,Bachelor's,1988.0,m,0.39,2012-10-23,2013-01-26 00:00:00.000000000,41.0,0,14.0,2349.0,0
6,627819,MITx/8.MReV/2013_Summer,MHxPC130254476,1,1,0,0,United States,Master's,1987.0,m,0.46,2013-06-19,2013-08-02 00:00:00.000000000,30.0,0,19.0,5208.0,0
7,75384,HarvardX/CS50x/2012,MHxPC130538825,1,1,0,0,Unknown/Other,Bachelor's,1986.0,m,0,2012-09-01,2012-10-23 03:56:11.943158893,2.0,0,1.0,1.0,1
8,536458,MITx/6.00x/2013_Spring,MHxPC130323295,1,1,0,0,India,Secondary,1993.0,m,0,2013-01-24,2013-05-15 00:00:00.000000000,13.0,0,6.0,170.0,0
9,438390,MITx/6.00x/2012_Fall,MHxPC130396038,1,1,0,0,United States,Bachelor's,1973.0,m,0,2012-09-10,2012-10-15 00:00:00.000000000,7.0,0,2.0,160.0,1


🔶 Prepare the split for the **[ Random Forest ]** model.

- This is with the small & balanced dataset we created earlier

In [ ]:
# Create a copy for this model execution
RF = small_balanced_data.copy()

In [ ]:
# Remove unnecessary features (faster execution)
drop_cols = ['final_cc_cname_DI', 'LoE_DI', 'YoB', 'start_time_DI', 'course_id',
             'userid_DI', 'index', 'last_event_DI']
RF.drop(columns=drop_cols, inplace=True)

In [ ]:
RF.dtypes

registered             int64
viewed                 int64
explored               int64
certified              int64
gender                object
grade                 object
ndays_act            float64
nforum_posts           int64
nchapters_imputed    float64
nevents_imputed      float64
dropout                int64
dtype: object

In [ ]:
# Separate features and target
X = RF.drop(columns=['dropout'])
y = RF['dropout']

# One-hot encode categorical features
X = pd.get_dummies(X, drop_first=True)

# Split the data (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
# Check for errors
X_train.dtypes.value_counts()

bool       104
int64        5
float64      3
Name: count, dtype: int64

🔶 **Random Forest**.

In [ ]:
# Initialize the model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
# Make predictions
y_pred = rf_model.predict(X_test)

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.988

Confusion Matrix:
 [[1994    6]
 [  42 1958]]

Classification Report:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99      2000
           1       1.00      0.98      0.99      2000

    accuracy                           0.99      4000
   macro avg       0.99      0.99      0.99      4000
weighted avg       0.99      0.99      0.99      4000



🔶 Cross-validation.

In [ ]:
# Check the correctness of the results
# Define multiple scorers
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score)
}

cv_results = cross_validate(rf_model, X, y, cv=5, scoring=scoring)

# Print average scores
print("Cross-Validation Results (5-fold):")
for metric in cv_results:
    if 'test' in metric:
        print(f"{metric}: {np.mean(cv_results[metric]):.4f}")

Cross-Validation Results (5-fold):
test_accuracy: 0.9887
test_precision: 0.9980
test_recall: 0.9793
test_f1: 0.9885


🔶 Prepare the split for the **[ Catboost ]** model.

In [ ]:
# Create a copy for this model execution
cb = small_balanced_data.copy()

In [ ]:
# Separate features and target
X = cb.drop(columns=['dropout'])
y = cb['dropout']

# Identify categorical features by name
cat_features = X.select_dtypes(include=['object', 'category']).columns.tolist()

# Split the data (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
# Check for errors
X_train.dtypes.value_counts()

object     8
int64      6
float64    4
Name: count, dtype: int64

🔶 **Catboost**.

In [ ]:
# Train CatBoost model with categorical features
cat_model = CatBoostClassifier(verbose=0, random_state=42)
cat_model.fit(X_train, y_train, cat_features=cat_features)

# Make predictions
y_pred = cat_model.predict(X_test)

# Evaluate performance
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("CatBoost on Balanced Data")
print(classification_report(y_test, y_pred))

Accuracy: 1.0

Confusion Matrix:
 [[2000    0]
 [   0 2000]]
CatBoost on Balanced Data
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2000
           1       1.00      1.00      1.00      2000

    accuracy                           1.00      4000
   macro avg       1.00      1.00      1.00      4000
weighted avg       1.00      1.00      1.00      4000



🔶 Cross-validation.

In [ ]:
# Identify categorical columns
cat_features = X.select_dtypes(include=['object', 'category']).columns.tolist()

# Convert these to 'category' dtype (optional but cleaner)
for col in cat_features:
    X[col] = X[col].astype('category')

# Define CV strategy
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Manual CV loop to allow passing cat_features
acc_scores, prec_scores, rec_scores, f1_scores = [], [], [], []

for train_idx, test_idx in cv.split(X, y):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    model = CatBoostClassifier(verbose=0, random_state=42)
    model.fit(X_train, y_train, cat_features=cat_features)

    y_pred = model.predict(X_test)

    acc_scores.append(accuracy_score(y_test, y_pred))
    prec_scores.append(precision_score(y_test, y_pred))
    rec_scores.append(recall_score(y_test, y_pred))
    f1_scores.append(f1_score(y_test, y_pred))

# Print the average results
print("CatBoost Classifier (5-Fold CV):")
print(f"Accuracy:  {np.mean(acc_scores):.4f}")
print(f"Precision: {np.mean(prec_scores):.4f}")
print(f"Recall:    {np.mean(rec_scores):.4f}")
print(f"F1-score:  {np.mean(f1_scores):.4f}")

CatBoost Classifier (5-Fold CV):
Accuracy:  0.9998
Precision: 0.9997
Recall:    0.9998
F1-score:  0.9998


🔶 Another method.

In [ ]:
# Define CatBoost model with balanced class weights
catboost_model = CatBoostClassifier(
    auto_class_weights='Balanced',
    random_seed=42,
    verbose=0,
    iterations=200  # Realistic number of iterations
)

# Hyperparameter grid for realistic search
param_grid_cb = {
    'depth': [4, 6],
    'learning_rate': [0.1, 0.05],
    'l2_leaf_reg': [3, 5]
}

# Setup GridSearchCV
grid_cb = GridSearchCV(
    catboost_model,
    param_grid_cb,
    cv=3,
    scoring='f1',
    n_jobs=-1,
    verbose=1
)

print("Training CatBoost...")
grid_cb.fit(X_train, y_train)
print("Training complete.")

# Predictions and evaluation
y_pred_cb = grid_cb.predict(X_test)

print("Best Parameters:", grid_cb.best_params_)
print("Best CV F1 Score:", grid_cb.best_score_)
print("\nAccuracy:", accuracy_score(y_test, y_pred_cb))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_cb))
print("\nClassification Report:\n", classification_report(y_test, y_pred_cb))

Training CatBoost...
Fitting 3 folds for each of 8 candidates, totalling 24 fits
Training complete.
Best Parameters: {'depth': 4, 'l2_leaf_reg': 5, 'learning_rate': 0.05}
Best CV F1 Score: 0.9996874804587385

Accuracy: 0.9995

Confusion Matrix:
 [[1998    2]
 [   0 2000]]

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      2000
           1       1.00      1.00      1.00      2000

    accuracy                           1.00      4000
   macro avg       1.00      1.00      1.00      4000
weighted avg       1.00      1.00      1.00      4000



🔶 Prepare the split for the **[ Gradient Boosting ]** model.

In [ ]:
# Create a copy for this model execution
gb = small_balanced_data.copy()

In [ ]:
# Separate features and target
X = gb.drop(columns=['dropout'])
y = gb['dropout']

# One-hot encode categorical features
X = pd.get_dummies(X, drop_first=True)

# Split the data (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
# Check for errors
X_train.dtypes.value_counts()

bool       20936
int64          6
float64        4
Name: count, dtype: int64

🔶 **Gradient Boosting**.

In [ ]:
gb_model = GradientBoostingClassifier(random_state=42)
gb_model.fit(X_train, y_train)

# Evaluation on balanced test set
y_pred = gb_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Gradient Boosting on Balanced Data")
print(classification_report(y_test, y_pred))

Accuracy: 1.0

Confusion Matrix:
 [[2000    0]
 [   0 2000]]
Gradient Boosting on Balanced Data
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2000
           1       1.00      1.00      1.00      2000

    accuracy                           1.00      4000
   macro avg       1.00      1.00      1.00      4000
weighted avg       1.00      1.00      1.00      4000



🔶 Cross-validation.

In [ ]:
# Cross-validation setup
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Metrics to report
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score)
}

In [ ]:
gb_model = GradientBoostingClassifier(random_state=42)

gb_scores = cross_validate(gb_model, X, y, cv=cv, scoring=scoring)

print("Gradient Boosting Classifier (5-Fold CV):")
print(f"Accuracy:  {np.mean(gb_scores['test_accuracy']):.4f}")
print(f"Precision: {np.mean(gb_scores['test_precision']):.4f}")
print(f"Recall:    {np.mean(gb_scores['test_recall']):.4f}")
print(f"F1-score:  {np.mean(gb_scores['test_f1']):.4f}")

Gradient Boosting Classifier (5-Fold CV):
Accuracy:  1.0000
Precision: 1.0000
Recall:    1.0000
F1-score:  1.0000


🔶 Another method.

In [ ]:
# Identify column types
categorical_cols = X_train.select_dtypes(include='object').columns.tolist()
numerical_cols = X_train.select_dtypes(include='number').columns.tolist()

# Define ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_cols),
        ('num', StandardScaler(), numerical_cols)
    ]
)

In [ ]:
# Pipeline: Preprocessing → SMOTE → Classifier
pipeline_gb = Pipeline([
    ('preprocessor', preprocessor),
    ('smote', SMOTE(random_state=42)),
    ('gb', GradientBoostingClassifier(random_state=42))
])

# Hyperparameter grid
param_grid_gb = {
    'gb__n_estimators': [100, 200],
    'gb__learning_rate': [0.1, 0.05],
    'gb__max_depth': [3, 5],
}

# GridSearchCV
grid_gb = GridSearchCV(
    pipeline_gb,
    param_grid=param_grid_gb,
    cv=5,
    scoring='f1',
    n_jobs=-1,
    verbose=2
)

# Fit on original raw data
print("Starting Gradient Boosting training...")
grid_gb.fit(X_train, y_train)

# Output best results
print("Gradient Boosting best params:", grid_gb.best_params_)
print("Gradient Boosting CV F1:", grid_gb.best_score_)

# Predict using the test set (automatically preprocessed)
y_pred_gb = grid_gb.predict(X_test)

print("\nGradient Boosting Classification Report:")
print(classification_report(y_test, y_pred_gb))

Starting Gradient Boosting training...
Fitting 5 folds for each of 8 candidates, totalling 40 fits
Gradient Boosting best params: {'gb__learning_rate': 0.1, 'gb__max_depth': 3, 'gb__n_estimators': 100}
Gradient Boosting CV F1: 1.0

Gradient Boosting Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2000
           1       1.00      1.00      1.00      2000

    accuracy                           1.00      4000
   macro avg       1.00      1.00      1.00      4000
weighted avg       1.00      1.00      1.00      4000



🔶 Prepare the split for the **[ Multi-layer Perceptron ]** model.

In [ ]:
# Create a copy for this model execution
mlp = small_balanced_data.copy()

In [ ]:
# Separate features and target
X = mlp.drop(columns=['dropout'])
y = mlp['dropout']

# One-hot encode categorical features
X = pd.get_dummies(X, drop_first=True)

# Split the data (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
# Check for errors
X_train.dtypes.value_counts()

bool       20936
int64          6
float64        4
Name: count, dtype: int64

🔶 **Multi-layer Perceptron (MLP)**.

In [ ]:
# Scale data for MLP
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

mlp_model = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=200, random_state=42)
mlp_model.fit(X_train_scaled, y_train)

# Evaluation on balanced test set
y_pred = mlp_model.predict(X_test_scaled)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("MLP on Balanced Data")
print(classification_report(y_test, y_pred))

Accuracy: 0.89875

Confusion Matrix:
 [[1840  160]
 [ 245 1755]]
MLP on Balanced Data
              precision    recall  f1-score   support

           0       0.88      0.92      0.90      2000
           1       0.92      0.88      0.90      2000

    accuracy                           0.90      4000
   macro avg       0.90      0.90      0.90      4000
weighted avg       0.90      0.90      0.90      4000



🔶 Another method.

In [ ]:
pipeline_mlp = Pipeline([
    ('smote', SMOTE(random_state=42)),
    ('mlp', MLPClassifier(random_state=42, max_iter=500))
])

param_grid_mlp = {
    'mlp__hidden_layer_sizes': [(50,), (100,)],
    'mlp__activation': ['relu', 'tanh'],
    'mlp__alpha': [0.0001, 0.001],  # L2 penalty (regularization)
}

grid_mlp = GridSearchCV(
    pipeline_mlp,
    param_grid_mlp,
    cv=5,
    scoring='f1',
    n_jobs=-1,
    verbose=2
)

grid_mlp.fit(X_train, y_train)

print("MLP best params:", grid_mlp.best_params_)
print("MLP CV F1:", grid_mlp.best_score_)

y_pred_mlp = grid_mlp.predict(X_test)
print(classification_report(y_test, y_pred_mlp))

Fitting 5 folds for each of 8 candidates, totalling 40 fits
MLP best params: {'mlp__activation': 'relu', 'mlp__alpha': 0.001, 'mlp__hidden_layer_sizes': (100,)}
MLP CV F1: 0.9973733371059982
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      2000
           1       0.98      1.00      0.99      2000

    accuracy                           0.99      4000
   macro avg       0.99      0.99      0.99      4000
weighted avg       0.99      0.99      0.99      4000

